In [2]:
# Install all the necessary python libraries if anaconda is not able to find them.
# !pip3 install google-maps-reviews
# Too bad the google-maps-reviews lib requires OutScraper API Key which is only free for the first 500 places and it requires minimum deposit of $500

In [72]:
# Import all the necessary python libraries
# import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [73]:
# Note: By default, The Safari WebDriver is included with the operating system. There is no additional installations required.
driver = webdriver.Safari()
driver.maximize_window()
url = 'https://www.google.com/maps'
driver.get(url)
time.sleep(5)

In [ ]:
# get the webpage
url = 'https://www.google.com/maps'
webpage = requests.get(url)
soup = BeautifulSoup(webpage.content, 'html.parser')

In [74]:
# Functions to return the element
def find_element_by_xpath(xpath):
    return driver.find_element(By.XPATH, xpath)

def find_element_by_css(css):
    return driver.find_element(By.CSS_SELECTOR, css)

In [ ]:
# Define what are we looking for in the query
FOOD_CATEGORY_LIST = ["halal food","western food","fast food","vegetarian food",
                      "chinese food","indian food","malay food","japanese food",
                      "korean food","cafe","bakery"]

DRINK_CATEGORY_LIST = ["coffee","bubble tea"]

LOCATION_LIST = ["Subang Jaya","Puchong","Sri Petaling","Bukit Jalil","Seri Kembangan","Bukit Bintang","Damansara","Kepong","Klang","Cheras"]
                 
# SEARCH_INPUT = f"{CATEGORY} near {LOCATION}"

In [75]:
# Locate the search element
search_box_element = find_element_by_xpath('//*[@id="searchboxinput"]')
search_box_element.send_keys(SEARCH_INPUT)
time.sleep(3)


<selenium.webdriver.remote.webelement.WebElement (session="022D59BE-59E5-4739-A769-0822D51485FD", element="node-C691E570-C4F1-48A5-A192-3506F64BB67B")>


In [76]:
search_button_element = find_element_by_xpath('//*[@id="searchbox-searchbutton"]')
search_button_element.click()


In [6]:
driver.quit()
